## Part 1: SQLite Database Operations for Vectors

### 1. Database Creation
- **Task:** Create a SQLite database.
- **Details:** 
  - Database Name: `vector-db.db`

### 2. Table Creation
- **Task:** Design and create a table schema for vector data.

### 3. Data Generation
- **Task:** Generate some vector data.

### 4. Data Serialization
- **Task:** Covret vector data to bytestream by sqlite3.Binary().

### 5. Data Insertion
- **Task:** Insert bytestream data into the table.

### 6. Data Retreival:
- **Task:** Execute SQL queries.

### 7. Data Deserializing
- **Task:** Convert back the bytestream data to the original vector object.

### 8. Closing Connection
- **Task:** Safely commit and close the database connection.

In [1]:
# !pip install sqlite3

import sqlite3
import numpy as np

In [2]:
# create a connection to the SQLite DB
conn = sqlite3.connect('vector-db.db')
# Create a cursor object to execute SQL Commands
cursor = conn.cursor()

In [3]:
# Create a table for vector data
cursor.execute(
"""
CREATE TABLE IF NOT EXISTS vectors (
    id INTEGER PRIMARY KEY,
    vector BLOB NOT NULL
)
"""
)

In [4]:
# generate some sample vectors
vect1 = np.array([1.2, 3.4, 2.1, 0.8])
vect2 = np.array([2.7, 1.5, 3.9, 2.3])

In [6]:
vect1.tobytes() # numpy array to bytestream

b'333333\xf3?333333\x0b@\xcd\xcc\xcc\xcc\xcc\xcc\x00@\x9a\x99\x99\x99\x99\x99\xe9?'

#### sqlite3.Binary: converts all the arrays/vectors to binary format

In [10]:
# Insert vector data into table
cursor.execute("INSERT INTO vectors (vector) VALUES (?)", 
               (sqlite3.Binary(vect1.tobytes()),))

In [11]:
cursor.execute("INSERT INTO vectors (vector) VALUES (?)", 
               (sqlite3.Binary(vect2.tobytes()),))

In [12]:
# Retreive data

cursor.execute("SELECT vector FROM vectors")

In [13]:
rows = cursor.fetchall()

In [14]:
rows # it contains two vectors

[(b'333333\xf3?333333\x0b@\xcd\xcc\xcc\xcc\xcc\xcc\x00@\x9a\x99\x99\x99\x99\x99\xe9?',),
 (b'\x9a\x99\x99\x99\x99\x99\x05@\x00\x00\x00\x00\x00\x00\xf8?333333\x0f@ffffff\x02@',)]

## Deserializing the retreived objects
Note: Now we have to do the reverse. So we have to convert back the byte stream to a numpy object

In [20]:
vector = np.frombuffer(rows[0][0], dtype = np.float64)

In [21]:
vector

array([1.2, 3.4, 2.1, 0.8])

In [22]:
vectors = []
for row in rows:
    vector = np.frombuffer(row[0], dtype = np.float64)
    vectors.append(vector)

In [23]:
vectors

[array([1.2, 3.4, 2.1, 0.8]), array([2.7, 1.5, 3.9, 2.3])]

# Part 2:  Vector Similarity Search (VSS)

### The goal here is to find a vector from the database that is closest to a query vector:

- **Creation of Query Vector:**
  - Developed a query vector for database operations.

- **Utilization of Vector Database:**
  - Employed a pre-existing set of vectors from the database.

- **Execution of SQL Statement:**
  - Ran `cursor.execute` to find the vector closest to the query vector.

- **Retrieval of Top Result:**
  - Used `cursor.fetchone()` to retrieve the top matching vector.

- **Conversion to Numpy Object:**
  - Transformed the retrieved data from buffer to original vector form using the correct data type in numpy.


In [24]:
query_vect = np.array([1.0, 3.2, 2.0, 0.5]) 

- ***Note 1***: The question marks in SQL is like passing a parameter, which is in this case id the query vector. So whatever the query vector is, basically it will be replaced by the question mark.

- ***Note 2***:Another important note is this search needs to be in the ascending order so the vector with the lowest distance should be at the top so we can actually pull the top one to find the closest vector

In [25]:
cursor.execute("""
SELECT vector FROM vectors ORDER BY abs(vector - ?) ASC
""", (sqlite3.Binary(query_vect.tobytes()),))

In [26]:
res = cursor.fetchone()  # finding the top one

- ***Note 3***: we don't have to do fetchall() here because we just want the top result which is the one vector closest to the query vector.

In [28]:
np.frombuffer(res[0], dtype=np.float64) # most similar vector

array([2.7, 1.5, 3.9, 2.3])

In [29]:
conn.commit()

In [30]:
conn.close()